In [18]:
import os

path = "/home/niceguy1575/2020/개인/economy_statsss"
if os.path.isdir(path) is not True:
    print("already exists")



already exists


In [21]:
not os.path.isdir(path)

True

In [24]:
import sys
sys.platform

'linux'

In [26]:
os.path.isdir(path)

False